# Project 3: Web APIs & Classification

#### By Bhupesh Kumar


#### Problem Statement :
In this project, I am going to exaplin how to take data from online sources using APIs and draw insights using Natural Language Processing (NLP).

Goal is to help engineers to understand the NLP and things can impact the prediction value. Baseline is to give them introduction to NLP procedure so they can look at cell phone market data using API's and make prediction.  

NLP helps us extract information from text and use it in out computations and algorithms. So in this project, I am going to extract the posts titles from reddit.com and make insights from it and prdict where the post came from. 


#### Approach 

    * Collect Data from subreddit(Iphone and Google Pixel) using the reddit API
    * Clean the data 
    * Model the data 

### Step 1- Data Collection 

##### Importing Libraries 

In [1]:
import pandas as pd
import numpy as np 
import requests
import time 

In [2]:
# URL for first subreddit (Iphone)

url_1 = 'https://www.reddit.com/r/iphone.json'

# URL for second subreddit (Google Pixel)

url_2 = 'https://www.reddit.com/r/GooglePixel.json'

# Custom User agent 
headers = {'User-agent': 'Bhupesh- Project 3'}

res_1 = requests.get(url_1, headers = headers)

res_2 = requests.get(url_2, headers = headers)

In [3]:
res_1.status_code

200

In [4]:
res_2.status_code

200

    Status code for both requests is 200 which is good.

### Collecting Posts from Both Subreddits 

####  SubReddit 1 - Iphone

In [5]:
iphone_json = res_1.json()

In [6]:
iphone_json.keys()

dict_keys(['kind', 'data'])

In [7]:
iphone_json['data'].keys()

dict_keys(['modhash', 'dist', 'children', 'after', 'before'])

In [8]:
iphone_json['data']['children'][1]

{'kind': 't3',
 'data': {'approved_at_utc': None,
  'subreddit': 'iphone',
  'selftext': '**Important Announcement**\n\nFollowing discussions amongst the mod team, all posts, videos, and image submissions will be filtered by AutoMod, from the point when this post goes live. This is due to the fact that the majority of the posts are of a tech support nature which is more appropriate in the Weekly Tech Support Thread. There are also many instances of spam, and other low-effort posts such as random off-topic posts or single sentence text posts, that are not conducive to a good discussion. As such, the rules of the subreddit has been clarified and are available on the sidebar or on this link here to [the wiki of the rules](https://www.reddit.com/r/iphone/wiki/rules).\n\nAs this sub grows in the number of subscribers, we are taking proactive steps to make this place more organized and neat for you, and we apologize for any inconvenience. You will receive a message when your submission have 

In [9]:
len(iphone_json['data']['children'])

27

Having prior knowledge about reddit API, I know all the information i need for this project is in ['data'] key of the json. The posts I am looking for are in ['children'] key of ['data']. 

Each pull give me about 27 posts but i need close to 1000 posts. I will create for loop to hit the API multiple times till have enough. I will do the range of 50 which will give me 1250-1300 posts. The reason I am pulling more post is because there are some redudent posts as well. For this project, I need unquie posts close to 1000. 

Also, I will use ['after'] key in data to set params. Which will allow me to me pull next 25 posts and so on.

In [10]:
iphone_posts = []
after = None

for i in range(50):
    if after == None:
        params = {}
    else:
        params = {'after':after}
        
    url_1 = 'https://www.reddit.com/r/iphone.json'
    res_1 = requests.get(url_1, params=params, headers = headers)
    
    if res_1.status_code == 200:
        iphone_json = res_1.json()
        iphone_posts.extend(iphone_json['data']['children'])
        after = iphone_json['data']['after']
    else:
        print(res.status_code)
        break
        
    time.sleep(1)

In [11]:
len(iphone_posts)

1230

In [12]:
len(set([p['data']['name']for p in iphone_posts]))# unique posts 

903

In [13]:
iphone_posts[0]['data']['title']

'Weekly Tech Support Thread - [July 09]'

#### SubReddit 2 - Google Pixel

In [14]:
google_pixel_posts = []
after = None

for i in range(50):
    if after == None:
        params = {}
    else:
        params = {'after':after}
        
    url_2 = 'https://www.reddit.com/r/GooglePixel.json'
    res_2 = requests.get(url_2, params=params, headers = headers)
    
    if res_2.status_code == 200:
        google_pixel_json = res_2.json()
        google_pixel_posts.extend(google_pixel_json['data']['children'])
        after = google_pixel_json['data']['after']
    else:
        print(res.status_code)
        break
        
    time.sleep(1)

In [15]:
len(google_pixel_posts)

1235

In [16]:
len(set([p['data']['name']for p in google_pixel_posts]))# unique posts 

908

In [17]:
google_pixel_posts[0]['data']['title']

'Weekly #MadeByGoogle Photos Megathread - July 04 2019'

    Iphone - 1230 posts, 903 unique
    Google pixel - 1235 posts, 908 unique

Now, I can collect the information i need for this project from those posts. 

#### IPHONE Post Title 
We are only interested in the text data data, so im going to extract the title of the post 

In [18]:
iphone_title = [iphone_posts[i]['data']['title'] for i in range(0,len(iphone_posts))]

Once i extracted the title, it is in list form. In order to run models on it, i will change it to pandas dataframe 

In [19]:
iphone_df = pd.DataFrame(iphone_title, columns=['post_title'])
iphone_df['rank'] = 'iphone'
iphone_df.head()

,post_title,rank
0,Weekly Tech Support Thread - [July 09],iphone
1,Important Announcement + Mod Applications,iphone
2,Well Why Not 😀,iphone
3,Paid apps actually worth it in 2019?,iphone
4,My iPhone collection as of yesterday! Just mis...,iphone


In [20]:
iphone_df.shape

(1230, 2)

#### GOOGLE PIXEL post title
I will do the same here will google pixel reddit, extract the title only. 

In [21]:
google_pixel_title = [google_pixel_posts[i]['data']['title'] for i in range(0,len(google_pixel_posts))]

Chagine the information into pandas dataframe

In [22]:
google_pixel_df = pd.DataFrame(google_pixel_title, columns=['post_title'])
google_pixel_df['rank'] = 'google_pixel'
google_pixel_df.head()

,post_title,rank
0,Weekly #MadeByGoogle Photos Megathread - July ...,google_pixel
1,"The July 2019 Superthread: #LeaksByGoogle, Bat...",google_pixel
2,Pixel 3 and 3 XL will be heavily discounted on...,google_pixel
3,Android Q Beta 5!,google_pixel
4,Q beta 5 assistant corner gesture. Let's talk ...,google_pixel


In [23]:
google_pixel_df.shape

(1235, 2)

#### Combining the two Dataframe into one

In [24]:
raw_df = pd.concat([iphone_df, google_pixel_df])

In [26]:
raw_df = raw_df.reset_index(drop = True)

In [27]:
raw_df.head()

,post_title,rank
0,Weekly Tech Support Thread - [July 09],iphone
1,Important Announcement + Mod Applications,iphone
2,Well Why Not 😀,iphone
3,Paid apps actually worth it in 2019?,iphone
4,My iPhone collection as of yesterday! Just mis...,iphone


In [28]:
raw_df.tail()

,post_title,rank
2460,Gboard without the stupid emoji button!,google_pixel
2461,Worth changing my OnePlus 5t to a Pixel 2xl,google_pixel
2462,"Google asked me... ""Are you ok?"" Today",google_pixel
2463,Can somebody take pictures of buttons on 2XL f...,google_pixel
2464,"Best Skins Ever review, buyer beware",google_pixel


In [29]:
raw_df.shape

(2465, 2)

#### Making rank column a boolean - Iphone (0) and google pixel (1)

I created a rank column above to identify post belongs to which reddit, but im going to change the rank into boolean form. It's easier for classification models having an numeric data. 

In [30]:
raw_df['rank'].value_counts()

google_pixel    1235
iphone          1230
Name: rank, dtype: int64

In [31]:
raw_df['rank'] = raw_df['rank'].map({'iphone':0,'google_pixel':1})

In [32]:
raw_df.head()

,post_title,rank
0,Weekly Tech Support Thread - [July 09],0
1,Important Announcement + Mod Applications,0
2,Well Why Not 😀,0
3,Paid apps actually worth it in 2019?,0
4,My iPhone collection as of yesterday! Just mis...,0


In [33]:
raw_df.tail()

,post_title,rank
2460,Gboard without the stupid emoji button!,1
2461,Worth changing my OnePlus 5t to a Pixel 2xl,1
2462,"Google asked me... ""Are you ok?"" Today",1
2463,Can somebody take pictures of buttons on 2XL f...,1
2464,"Best Skins Ever review, buyer beware",1


### Deleting Dublicate data rows

There are only 905 iphone unique rows, and 905 google pixel rows. 

In [34]:
raw_df.drop_duplicates(subset ="post_title", 
                     keep = 'first', inplace = True) 

In [35]:
raw_df.shape

(1807, 2)

#### Exporting the data for cleaning and modeling. 

In [36]:
raw_df.to_csv('raw_title_dataset')